#### 참고 
###### cifar-10 dataset 원본 위치: /media/data2/hyunjun/content/origin-cifar-10
###### cifar-10 dataset svd 처리한 거 위치: /media/data2/hyunjun/content/svd-cifar-10

In [16]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
from torchvision.io import read_image
from torchvision import transforms
from torchvision import models
import os

from efficientnet_pytorch import EfficientNet

In [17]:
lr = 1e-3
batch_size = 64

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

trans = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.243, 0.262))
])

root = '' ## 이미지 경로

original_trainset = datasets.ImageFolder(root=root+"/origin-cifar-10/train", transform=trans)
original_testset = datasets.ImageFolder(root=root+"origin-cifar-10/test", transform=trans)
svd_trainset = datasets.ImageFolder(root=root+"/svd-cifar-10/train", transform=trans)
svd_testset = datasets.ImageFolder(root=root+"/svd-cifar-10/test", transform=trans)
svd4_trainset = datasets.ImageFolder(root=root+"/svd4-cifar-10/train", transform=trans)
svd4_testset = datasets.ImageFolder(root=root+"/svd4-cifar-10/test", transform=trans)

o_trainloader = DataLoader(original_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
o_testloader = DataLoader(original_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s_trainloader = DataLoader(svd_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s_testloader = DataLoader(svd_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s4_trainloader = DataLoader(svd4_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s4_testloader = DataLoader(svd4_testset, batch_size=batch_size, shuffle=False, drop_last=False)

classes = original_testset.classes

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model1 = EfficientNet.from_pretrained('efficientnet-b0')
model2 = EfficientNet.from_pretrained('efficientnet-b0')
model3 = EfficientNet.from_pretrained('efficientnet-b0')

model1._fc = nn.Linear(in_features=model1._fc.in_features, out_features=10)
model2._fc = nn.Linear(in_features=model2._fc.in_features, out_features=10)
model3._fc = nn.Linear(in_features=model3._fc.in_features, out_features=10)
nn.init.xavier_normal_(model1._fc.weight)
nn.init.xavier_normal_(model2._fc.weight)
nn.init.xavier_normal_(model3._fc.weight)

model1.to(device)
model2.to(device)
model3.to(device)

Using cuda device
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

def train_loop(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    loss_sum = 0
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X.to(device))
        loss = criterion(pred, y.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            loss_sum += loss
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

    return loss_sum / size

def test_loop(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    hit, loss = 0, 0
    conf_matrix = []
    for i in range(10):
        conf_matrix.append([0, 0, 0, 0])

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            loss += criterion(pred, y.to(device)).item()
            hit += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()
            temp = pred.argmax(1)
            for k in range(len(temp)):
                for i in range(10):
                    if i == temp[k]:
                        if i == y[k]:
                            conf_matrix[i][0] += 1.0
                        else:
                            conf_matrix[i][1] += 1.0
                    else:
                        if i == y[k]:
                            conf_matrix[i][2] += 1.0
                        else:
                            conf_matrix[i][3] += 1.0
    
    loss /= (size/batch_size)
    hit /= size
    print(f"Test Error: \n Accuracy: {(100*hit):>0.1f}%, Avg loss: {loss:>8f}\n")

    return conf_matrix

In [19]:
epochs = 20
train_loss1, test_res1, test_res2, test_res3 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model1.train()
    train_loss1.append(train_loop(o_trainloader, model1, criterion, optimizer1))
    model1.eval()
    test_res1.append(test_loop(o_testloader, model1, criterion))
    test_res2.append(test_loop(s_testloader, model1, criterion))
    test_res3.append(test_loop(s4_testloader, model1, criterion))

Epoch 1-------------------------------
loss: 2.903020 [    0/50000]
loss: 1.828581 [ 6400/50000]
loss: 1.285742 [12800/50000]
loss: 1.086572 [19200/50000]
loss: 1.021788 [25600/50000]
loss: 0.766840 [32000/50000]
loss: 1.053449 [38400/50000]
loss: 0.902497 [44800/50000]
Test Error: 
 Accuracy: 75.2%, Avg loss: 0.739391

Test Error: 
 Accuracy: 71.0%, Avg loss: 0.841636

Test Error: 
 Accuracy: 53.5%, Avg loss: 1.434585

Epoch 2-------------------------------
loss: 0.707691 [    0/50000]
loss: 1.097934 [ 6400/50000]
loss: 0.498116 [12800/50000]
loss: 0.790496 [19200/50000]
loss: 0.565366 [25600/50000]
loss: 0.645308 [32000/50000]
loss: 0.888303 [38400/50000]
loss: 0.340562 [44800/50000]
Test Error: 
 Accuracy: 77.2%, Avg loss: 0.696730

Test Error: 
 Accuracy: 71.2%, Avg loss: 0.883601

Test Error: 
 Accuracy: 48.4%, Avg loss: 1.732834

Epoch 3-------------------------------
loss: 0.528202 [    0/50000]
loss: 0.426532 [ 6400/50000]
loss: 0.540750 [12800/50000]
loss: 0.570562 [19200/5000

In [20]:
epochs = 20
train_loss2, test_res4, test_res5, test_res6 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model2.train()
    train_loss2.append(train_loop(s_trainloader, model2, criterion, optimizer2))
    model2.eval()
    test_res4.append(test_loop(o_testloader, model2, criterion))
    test_res5.append(test_loop(s_testloader, model2, criterion))
    test_res6.append(test_loop(s4_testloader, model2, criterion))

Epoch 1-------------------------------
loss: 2.824425 [    0/50000]
loss: 1.521721 [ 6400/50000]
loss: 1.153502 [12800/50000]
loss: 1.204732 [19200/50000]
loss: 1.122265 [25600/50000]
loss: 1.076364 [32000/50000]
loss: 0.736479 [38400/50000]
loss: 0.862050 [44800/50000]
Test Error: 
 Accuracy: 69.9%, Avg loss: 0.882837

Test Error: 
 Accuracy: 70.9%, Avg loss: 0.836010

Test Error: 
 Accuracy: 58.1%, Avg loss: 1.228809

Epoch 2-------------------------------
loss: 0.575866 [    0/50000]
loss: 0.868848 [ 6400/50000]
loss: 0.914497 [12800/50000]
loss: 0.844224 [19200/50000]
loss: 0.732538 [25600/50000]
loss: 0.769383 [32000/50000]
loss: 0.924691 [38400/50000]
loss: 0.537381 [44800/50000]
Test Error: 
 Accuracy: 74.9%, Avg loss: 0.756354

Test Error: 
 Accuracy: 75.8%, Avg loss: 0.714184

Test Error: 
 Accuracy: 61.4%, Avg loss: 1.200454

Epoch 3-------------------------------
loss: 0.513569 [    0/50000]
loss: 0.622715 [ 6400/50000]
loss: 0.861878 [12800/50000]
loss: 0.693696 [19200/5000

In [21]:
epochs = 20
train_loss3, test_res7, test_res8, test_res9 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model3.train()
    train_loss3.append(train_loop(s4_trainloader, model3, criterion, optimizer3))
    model3.eval()
    test_res7.append(test_loop(o_testloader, model3, criterion))
    test_res8.append(test_loop(s_testloader, model3, criterion))
    test_res9.append(test_loop(s4_testloader, model3, criterion))

Epoch 1-------------------------------
loss: 2.601522 [    0/50000]
loss: 1.739254 [ 6400/50000]
loss: 1.475773 [12800/50000]
loss: 1.229420 [19200/50000]
loss: 0.995757 [25600/50000]
loss: 1.222319 [32000/50000]
loss: 1.214512 [38400/50000]
loss: 0.989437 [44800/50000]
Test Error: 
 Accuracy: 58.9%, Avg loss: 1.231928

Test Error: 
 Accuracy: 63.3%, Avg loss: 1.073367

Test Error: 
 Accuracy: 63.8%, Avg loss: 1.042639

Epoch 2-------------------------------
loss: 1.002802 [    0/50000]
loss: 0.923861 [ 6400/50000]
loss: 0.862455 [12800/50000]
loss: 1.116309 [19200/50000]
loss: 0.816272 [25600/50000]
loss: 1.049231 [32000/50000]
loss: 0.791166 [38400/50000]
loss: 0.768443 [44800/50000]
Test Error: 
 Accuracy: 61.9%, Avg loss: 1.235885

Test Error: 
 Accuracy: 68.0%, Avg loss: 0.974167

Test Error: 
 Accuracy: 68.8%, Avg loss: 0.898509

Epoch 3-------------------------------
loss: 0.837048 [    0/50000]
loss: 0.670986 [ 6400/50000]
loss: 0.666613 [12800/50000]
loss: 0.510436 [19200/5000

In [22]:
print("Result for Testing with original datasets")
print(max(test_res1), max(test_res2), max(test_res3))

print("Result for Testing with original datasets")
print(max(test_res4), max(test_res5), max(test_res6))

print("Result for Testing with original datasets")
print(max(test_res7), max(test_res8), max(test_res9))

Result for Testing with original datasets
[[895.0, 326.0, 105.0, 8674.0], [914.0, 140.0, 86.0, 8860.0], [718.0, 279.0, 282.0, 8721.0], [614.0, 328.0, 386.0, 8672.0], [712.0, 194.0, 288.0, 8806.0], [675.0, 205.0, 325.0, 8795.0], [856.0, 195.0, 144.0, 8805.0], [839.0, 158.0, 161.0, 8842.0], [850.0, 79.0, 150.0, 8921.0], [868.0, 155.0, 132.0, 8845.0]] [[855.0, 252.0, 145.0, 8748.0], [782.0, 70.0, 218.0, 8930.0], [721.0, 280.0, 279.0, 8720.0], [615.0, 415.0, 385.0, 8585.0], [733.0, 266.0, 267.0, 8734.0], [699.0, 372.0, 301.0, 8628.0], [700.0, 86.0, 300.0, 8914.0], [858.0, 200.0, 142.0, 8800.0], [874.0, 119.0, 126.0, 8881.0], [871.0, 232.0, 129.0, 8768.0]] [[636.0, 293.0, 364.0, 8707.0], [368.0, 31.0, 632.0, 8969.0], [670.0, 616.0, 330.0, 8384.0], [535.0, 825.0, 465.0, 8175.0], [631.0, 531.0, 369.0, 8469.0], [446.0, 320.0, 554.0, 8680.0], [149.0, 15.0, 851.0, 8985.0], [667.0, 267.0, 333.0, 8733.0], [856.0, 597.0, 144.0, 8403.0], [809.0, 738.0, 191.0, 8262.0]]
Result for Testing with origina

In [23]:
def best_acc_finder(test_res):
    best, max = 0, 0
    for i in range(len(test_res)):
        correct = 0.0
        total = 0.0
        for j in range(len(test_res[i])):
            correct = test_res[i][j][0] +  test_res[i][j][3]
            total = correct + test_res[i][j][2] + test_res[i][j][1]
            if max < (correct / total):
                max = correct / total
                best = i

    print(f"best accuary: {max:>0.2f} at {best}")
    return best

In [24]:
best_acc_finder(test_res1)
best_acc_finder(test_res2)
best_acc_finder(test_res3)
best_acc_finder(test_res4)
best_acc_finder(test_res5)
best_acc_finder(test_res6)
best_acc_finder(test_res7)
best_acc_finder(test_res8)
best_acc_finder(test_res9)

best accuary: 0.98 at 15
best accuary: 0.98 at 15
best accuary: 0.95 at 2
best accuary: 0.98 at 14
best accuary: 0.98 at 15
best accuary: 0.96 at 7
best accuary: 0.97 at 13
best accuary: 0.97 at 13
best accuary: 0.97 at 13


13

In [25]:
def metrics_single_class(conf_mat):
    accuracy = (conf_mat[0] + conf_mat[3]) / (conf_mat[0] + conf_mat[1] + conf_mat[2] + conf_mat[3])
    precision = conf_mat[0] / (conf_mat[0] + conf_mat[1])
    recall = conf_mat[0] / (conf_mat[0] + conf_mat[2])
    f1_score = 2 * precision * recall / (precision + recall)
    
    return accuracy, precision, recall, f1_score

def metrics(conf_mats):
    sum_acc, sum_pre, sum_recall, sum_f1 = 0.0, 0.0, 0.0, 0.0
    for i in range(len(conf_mats)):
        temp_acc, temp_pre, temp_recall, temp_f1 = metrics_single_class(conf_mats[i])
        #print(f"For class-{i}:\n Accuracy: {temp_acc:>0.4f}, Precision: {temp_pre:>0.4f}, Recall: {temp_recall:>0.4f}, F1-score: {temp_f1:>0.4f}")
        sum_acc += temp_acc
        sum_pre += temp_pre
        sum_recall += temp_recall
        sum_f1 += temp_f1

    sum_acc /= 10
    sum_pre /= 10
    sum_recall /= 10
    sum_f1 /= 10

    return sum_acc, sum_pre, sum_recall, sum_f1


print(metrics(test_res1[best_acc_finder(test_res1)]))
print(metrics(test_res2[best_acc_finder(test_res2)]))
print(metrics(test_res3[best_acc_finder(test_res3)]))
print(metrics(test_res4[best_acc_finder(test_res4)]))
print(metrics(test_res5[best_acc_finder(test_res5)]))
print(metrics(test_res6[best_acc_finder(test_res6)]))
print(metrics(test_res7[best_acc_finder(test_res7)]))
print(metrics(test_res8[best_acc_finder(test_res8)]))
print(metrics(test_res9[best_acc_finder(test_res9)]))

best accuary: 0.98 at 15
(0.9616199999999999, 0.8187222334212299, 0.8080999999999999, 0.8098547825669309)
best accuary: 0.98 at 15
(0.95404, 0.7869108906626759, 0.7702, 0.773668695641539)
best accuary: 0.95 at 2
(0.9166000000000001, 0.6544054866156535, 0.583, 0.5664532632586912)
best accuary: 0.98 at 14
(0.9577199999999999, 0.7977390132012483, 0.7886, 0.7889599638747092)
best accuary: 0.98 at 15
(0.95666, 0.7948065138701295, 0.7833, 0.7860066290166191)
best accuary: 0.96 at 7
(0.9317599999999999, 0.6905009029443699, 0.6588, 0.6577239086128633)
best accuary: 0.97 at 13
(0.9429599999999999, 0.7365213162742352, 0.7148000000000001, 0.7112990878830537)
best accuary: 0.97 at 13
(0.9476800000000001, 0.7500167013281633, 0.7384000000000001, 0.7364707890769435)
best accuary: 0.97 at 13
(0.9465, 0.7423803817375129, 0.7325, 0.7344577472568428)
